<h2>PyTorch Implementation of the Workshop Notebook</h2>

PyTorch alternative for the classification with deep learning section of workshop notebook.

In [4]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models

<h3>Dataset</h3>

In [5]:
images_dir = r'..\dataset'

In [6]:
categories = os.listdir(images_dir)
m = 0
for category in categories:
    category_dir = os.path.join(images_dir, category)
    class_size = len(os.listdir(category_dir))
    print('Images belonging to class "', category, '":', class_size)
    m += class_size
print('We have ', m, ' images') # 1800

Images belonging to class " crazing ": 180
Images belonging to class " inclusion ": 180
Images belonging to class " patches ": 180
Images belonging to class " pitted_surface ": 180
Images belonging to class " rolled-in_scale ": 180
Images belonging to class " scratches ": 180
We have  1080  images


In [7]:
train_dir = r'..\train'
val_dir = r'..\val'
os.mkdir(val_dir)
test_dir = r'..\test'
os.mkdir(test_dir)

In [5]:
print(categories)
print(len(categories), 'classes')

['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']
6 classes


In [6]:
for category in categories:
    path = os.path.join(val_dir, category)
    os.mkdir(path)
    path = os.path.join(test_dir, category)
    os.mkdir(path)

Splitting 20% of total to validation set and another 20% for the test set, as done in original notebook. Since we have 6 classes, and we want to split this 20 percents without creating imbalances in test and validation sets, we take `(m*.2)/6`-image from each class.

In [7]:
split = int((m*.2)//6)

In [8]:
for class_name in categories:
    src_dir = os.path.join(images_dir, class_name)
    imgs_to_relocate = os.listdir(src_dir)[:split]
    for image in imgs_to_relocate:
        src = os.path.join(src_dir, image)
        dst = os.path.join(test_dir, class_name, image)
        shutil.move(src, dst)

In [10]:
for class_name in categories:
    src_dir = os.path.join(images_dir, class_name)
    imgs_to_relocate = os.listdir(src_dir)[:split]
    for image in imgs_to_relocate:
        src = os.path.join(src_dir, image)
        dst = os.path.join(val_dir, class_name, image)
        shutil.move(src, dst)

Checking sample sizes:

In [11]:
for category in categories:
    category_dir = os.path.join(test_dir, category)
    class_size = len(os.listdir(category_dir))
    print('In test set, images belonging to class "', category, '":', class_size)

In test set, images belonging to class " crazing ": 60
In test set, images belonging to class " inclusion ": 60
In test set, images belonging to class " patches ": 60
In test set, images belonging to class " pitted_surface ": 60
In test set, images belonging to class " rolled-in_scale ": 60
In test set, images belonging to class " scratches ": 60


In [12]:
for category in categories:
    category_dir = os.path.join(val_dir, category)
    class_size = len(os.listdir(category_dir))
    print('In validation set, images belonging to class "', category, '":', class_size)

In validation set, images belonging to class " crazing ": 60
In validation set, images belonging to class " inclusion ": 60
In validation set, images belonging to class " patches ": 60
In validation set, images belonging to class " pitted_surface ": 60
In validation set, images belonging to class " rolled-in_scale ": 60
In validation set, images belonging to class " scratches ": 60


Now decreased dataset file contains our training samples

In [8]:
train_dir = r'..\dataset'
for category in categories:
    category_dir = os.path.join(train_dir, category)
    class_size = len(os.listdir(category_dir))
    print('In training set, images belonging to class "', category, '":', class_size)

In training set, images belonging to class " crazing ": 180
In training set, images belonging to class " inclusion ": 180
In training set, images belonging to class " patches ": 180
In training set, images belonging to class " pitted_surface ": 180
In training set, images belonging to class " rolled-in_scale ": 180
In training set, images belonging to class " scratches ": 180


Image examples:

In [9]:
def process(filename: str=None) -> None:
    """
    View multiple images stored in files, stacking vertically

    Arguments:
        filename: str - path to filename containing image
    """
    image = mpimg.imread(filename)
    # <something gets done here>
    plt.figure()
    plt.imshow(image)

In [10]:
# sample_images_list = [r'..\test\scratches\scratches_1.jpg',
#                       r'..\test\scratches\inclusion_1.jpg',
#                       r'..\test\scratches\patches_1.jpg']

# for image in sample_images_list:
#     process(image)

NOTE: Dead kernel error after running Matplotlib is a current problem: https://stackoverflow.com/questions/69786885/after-conda-update-python-kernel-crashes-when-matplotlib-is-used. You can take a look at the samples from the repo.

<h3>Preprocessing</h3>

`transforms.ToTensor()` transforms image to tensor with floats as well as rescaling pixel values between 0 and 255 to 0 and 1 (by simply dividing tensors with 255).

In [11]:
transforms = transforms.Compose([transforms.Resize(100), # image size is 100 x 100 as determined in original notebook
                                 transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                             std=[0.229, 0.224, 0.225])])

If you want to normalize images, PyTorch needs mean and standard deviation of each channel of images as parameters for `transforms.Normalize()`. Instead of computing the values specific for your dataset, you can use values of ImageNet dataset.

In [12]:
train_data = torchvision.datasets.ImageFolder(root=train_dir, transform=transforms)
val_data = torchvision.datasets.ImageFolder(root=val_dir, transform=transforms)
test_data = torchvision.datasets.ImageFolder(root=test_dir, transform=transforms)

Finally, we'll create `DataLoader` instances that are appropriate type of input for PyTorch neural networks.

In [13]:
batch_size = 8 # as determined in original notebook
train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader = DataLoader(val_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

<h3>Modeling</h3>

Some remarks:
1. TensorFlow's `layers.MaxPooling2D()` has `pool_size=(2,2)` as default, while PyTorch has no default value for `kernel_size` (they determine the same thing).
2. Implementation of a convolutional layer: In TF, arguments are `(filters, kernel_size, ...)`; in PyTorch, it's `(in_channels, out_channels, ...)`.
3. A convention difference between these two libraries is about the order of dimensions: In TF, input tensor is of shape `(batch_dim, height, width, channels)` while in PyTorch the shape is `(batch_dim, channels, height, width)`.
4. `torch.nn.CrossEntropyLoss()` utilizes the softmax function before computing the loss.

In [14]:
class CNNNet(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 16, kernel_size=3, padding='same'), nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2),
                                      
                                      nn.Conv2d(16, 32, kernel_size=3, padding='same'), nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2),
                                      
                                      nn.Conv2d(32, 64, kernel_size=3, padding='same'), nn.ReLU(),
                                      nn.MaxPool2d(kernel_size=2))
        self.classifier = nn.Sequential(nn.Linear(9216, 128), nn.ReLU(),
                                        nn.Linear(128, 6)) # classification layer
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1) # flattening the output of convolution part of the network
        x = self.classifier(x)
        return x

Now we define a function to do training and computing loss (model will be validated on validation data at the end of each epoch, which is a process also visible in TF's history object's print.) Before all, in order to utilize GPU, we have to define an object like below and save models and tensors to the GPU whenever we're going to work with one.

In [15]:
device = torch.device("cuda")

In [16]:
cnn0 = CNNNet()

Setting the optimizer (will pass a different value to learning rate because original one results in flat learning curve.)

In [17]:
optimizer0 = optim.Adam(cnn0.parameters(), lr=1e-3)

In [18]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20):
    # training
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for inputs, targets in train_loader: # for a batch
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device) # utilizing the GPU
            output = model(inputs) # obtaining outputs of the network
            loss = loss_fn(output, targets) # computing loss
            loss.backward() # computing the gradients
            optimizer.step() # using gradients for optimization of parameters
            training_loss += loss.data.item()
        training_loss /= len(train_loader) # computing training loss at the end of an epoch
    
        model.eval()
        correct = 0
        total = 0

        # validation
        for inputs, targets in val_loader: # for a batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets)
            valid_loss += loss.data.item()
            # computing accuracy
            _, y_pred_tags = output.max(1)
            total += targets.size(0)
            correct += y_pred_tags.eq(targets).sum().item()
            acc = 100.*correct/total
        valid_loss /= len(val_loader)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, Accuracy: {:.2f}'.format(epoch, training_loss,
                                                                                                   valid_loss, acc))

In [19]:
train(cnn0, optimizer0, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader)

Epoch: 0, Training Loss: 3.09, Validation Loss: 1.88, Accuracy: 23.89
Epoch: 1, Training Loss: 1.81, Validation Loss: 4.84, Accuracy: 16.67
Epoch: 2, Training Loss: 2.06, Validation Loss: 1.80, Accuracy: 16.67
Epoch: 3, Training Loss: 1.82, Validation Loss: 1.80, Accuracy: 16.67
Epoch: 4, Training Loss: 1.84, Validation Loss: 1.79, Accuracy: 16.67
Epoch: 5, Training Loss: 1.81, Validation Loss: 1.79, Accuracy: 16.67
Epoch: 6, Training Loss: 1.81, Validation Loss: 1.79, Accuracy: 16.67
Epoch: 7, Training Loss: 1.81, Validation Loss: 1.79, Accuracy: 16.67
Epoch: 8, Training Loss: 1.80, Validation Loss: 1.82, Accuracy: 16.67
Epoch: 9, Training Loss: 1.75, Validation Loss: 1.80, Accuracy: 16.67
Epoch: 10, Training Loss: 2.01, Validation Loss: 10.93, Accuracy: 16.67
Epoch: 11, Training Loss: 2.01, Validation Loss: 21.31, Accuracy: 16.67
Epoch: 12, Training Loss: 1.86, Validation Loss: 1.94, Accuracy: 16.67
Epoch: 13, Training Loss: 1.83, Validation Loss: 26.89, Accuracy: 16.67
Epoch: 14, Tr

In [20]:
cnn1 = CNNNet()

In [21]:
optimizer1 = optim.Adam(cnn1.parameters(), lr=5e-5)

In [22]:
train(cnn1, optimizer1, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20)

Epoch: 0, Training Loss: 1.89, Validation Loss: 1.76, Accuracy: 27.22
Epoch: 1, Training Loss: 1.78, Validation Loss: 1.70, Accuracy: 30.28
Epoch: 2, Training Loss: 1.76, Validation Loss: 1.62, Accuracy: 27.22
Epoch: 3, Training Loss: 1.72, Validation Loss: 1.57, Accuracy: 25.83
Epoch: 4, Training Loss: 1.74, Validation Loss: 1.55, Accuracy: 43.89
Epoch: 5, Training Loss: 1.70, Validation Loss: 1.50, Accuracy: 40.56
Epoch: 6, Training Loss: 1.69, Validation Loss: 1.44, Accuracy: 45.00
Epoch: 7, Training Loss: 1.63, Validation Loss: 1.40, Accuracy: 48.33
Epoch: 8, Training Loss: 1.61, Validation Loss: 1.35, Accuracy: 50.28
Epoch: 9, Training Loss: 1.57, Validation Loss: 1.30, Accuracy: 53.33
Epoch: 10, Training Loss: 1.54, Validation Loss: 1.25, Accuracy: 58.33
Epoch: 11, Training Loss: 1.49, Validation Loss: 1.19, Accuracy: 61.94
Epoch: 12, Training Loss: 1.44, Validation Loss: 1.10, Accuracy: 67.50
Epoch: 13, Training Loss: 1.35, Validation Loss: 1.02, Accuracy: 70.56
Epoch: 14, Train

As we're underfitting, we should increase number of epochs and train again. We can continue from where we left off.

In [25]:
train(cnn1, optimizer1, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20)

Epoch: 0, Training Loss: 0.90, Validation Loss: 0.67, Accuracy: 83.89
Epoch: 1, Training Loss: 0.85, Validation Loss: 0.64, Accuracy: 85.28
Epoch: 2, Training Loss: 0.81, Validation Loss: 0.61, Accuracy: 85.28
Epoch: 3, Training Loss: 0.77, Validation Loss: 0.59, Accuracy: 84.17
Epoch: 4, Training Loss: 0.73, Validation Loss: 0.58, Accuracy: 84.44
Epoch: 5, Training Loss: 0.69, Validation Loss: 0.57, Accuracy: 84.17
Epoch: 6, Training Loss: 0.65, Validation Loss: 0.55, Accuracy: 83.33
Epoch: 7, Training Loss: 0.62, Validation Loss: 0.54, Accuracy: 83.06
Epoch: 8, Training Loss: 0.59, Validation Loss: 0.52, Accuracy: 84.17
Epoch: 9, Training Loss: 0.56, Validation Loss: 0.50, Accuracy: 83.89
Epoch: 10, Training Loss: 0.54, Validation Loss: 0.48, Accuracy: 84.44
Epoch: 11, Training Loss: 0.52, Validation Loss: 0.50, Accuracy: 82.22
Epoch: 12, Training Loss: 0.50, Validation Loss: 0.44, Accuracy: 85.56
Epoch: 13, Training Loss: 0.48, Validation Loss: 0.42, Accuracy: 86.39
Epoch: 14, Train

In [26]:
train(cnn1, optimizer1, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20)

Epoch: 0, Training Loss: 0.34, Validation Loss: 0.26, Accuracy: 92.78
Epoch: 1, Training Loss: 0.32, Validation Loss: 0.25, Accuracy: 93.89
Epoch: 2, Training Loss: 0.31, Validation Loss: 0.24, Accuracy: 94.17
Epoch: 3, Training Loss: 0.30, Validation Loss: 0.23, Accuracy: 94.44
Epoch: 4, Training Loss: 0.29, Validation Loss: 0.22, Accuracy: 94.72
Epoch: 5, Training Loss: 0.28, Validation Loss: 0.21, Accuracy: 95.00
Epoch: 6, Training Loss: 0.27, Validation Loss: 0.21, Accuracy: 94.17
Epoch: 7, Training Loss: 0.27, Validation Loss: 0.19, Accuracy: 95.00
Epoch: 8, Training Loss: 0.25, Validation Loss: 0.19, Accuracy: 94.72
Epoch: 9, Training Loss: 0.25, Validation Loss: 0.17, Accuracy: 94.72
Epoch: 10, Training Loss: 0.23, Validation Loss: 0.17, Accuracy: 94.72
Epoch: 11, Training Loss: 0.24, Validation Loss: 0.16, Accuracy: 94.72
Epoch: 12, Training Loss: 0.22, Validation Loss: 0.15, Accuracy: 95.56
Epoch: 13, Training Loss: 0.22, Validation Loss: 0.14, Accuracy: 96.11
Epoch: 14, Train

In [27]:
train(cnn1, optimizer1, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20)

Epoch: 0, Training Loss: 0.16, Validation Loss: 0.13, Accuracy: 96.67
Epoch: 1, Training Loss: 0.17, Validation Loss: 0.14, Accuracy: 96.67
Epoch: 2, Training Loss: 0.16, Validation Loss: 0.15, Accuracy: 96.67
Epoch: 3, Training Loss: 0.16, Validation Loss: 0.16, Accuracy: 95.83
Epoch: 4, Training Loss: 0.16, Validation Loss: 0.14, Accuracy: 96.67
Epoch: 5, Training Loss: 0.15, Validation Loss: 0.15, Accuracy: 95.83
Epoch: 6, Training Loss: 0.15, Validation Loss: 0.15, Accuracy: 95.83
Epoch: 7, Training Loss: 0.15, Validation Loss: 0.15, Accuracy: 95.28
Epoch: 8, Training Loss: 0.15, Validation Loss: 0.15, Accuracy: 95.83
Epoch: 9, Training Loss: 0.14, Validation Loss: 0.14, Accuracy: 95.83
Epoch: 10, Training Loss: 0.15, Validation Loss: 0.14, Accuracy: 96.39
Epoch: 11, Training Loss: 0.13, Validation Loss: 0.15, Accuracy: 95.28
Epoch: 12, Training Loss: 0.14, Validation Loss: 0.15, Accuracy: 95.28
Epoch: 13, Training Loss: 0.13, Validation Loss: 0.15, Accuracy: 95.00
Epoch: 14, Train

After 60 epochs, model start to overfit training set.

In [28]:
cnn2 = CNNNet()
optimizer2 = optim.Adam(cnn2.parameters(), lr=5e-5)
train(cnn2, optimizer2, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=60)

Epoch: 0, Training Loss: 2.04, Validation Loss: 1.76, Accuracy: 16.67
Epoch: 1, Training Loss: 1.79, Validation Loss: 1.72, Accuracy: 26.67
Epoch: 2, Training Loss: 1.75, Validation Loss: 1.65, Accuracy: 32.78
Epoch: 3, Training Loss: 1.75, Validation Loss: 1.58, Accuracy: 32.22
Epoch: 4, Training Loss: 1.75, Validation Loss: 1.58, Accuracy: 32.22
Epoch: 5, Training Loss: 1.74, Validation Loss: 1.54, Accuracy: 30.00
Epoch: 6, Training Loss: 1.70, Validation Loss: 1.50, Accuracy: 44.44
Epoch: 7, Training Loss: 1.68, Validation Loss: 1.44, Accuracy: 47.78
Epoch: 8, Training Loss: 1.64, Validation Loss: 1.39, Accuracy: 50.56
Epoch: 9, Training Loss: 1.61, Validation Loss: 1.35, Accuracy: 52.50
Epoch: 10, Training Loss: 1.58, Validation Loss: 1.30, Accuracy: 57.22
Epoch: 11, Training Loss: 1.54, Validation Loss: 1.24, Accuracy: 59.17
Epoch: 12, Training Loss: 1.48, Validation Loss: 1.19, Accuracy: 61.11
Epoch: 13, Training Loss: 1.44, Validation Loss: 1.14, Accuracy: 62.50
Epoch: 14, Train

In [29]:
torch.save(cnn2, r'..\cnn')

In [30]:
cnn = torch.load(r'..\cnn')

<h2>Final Evaluation</h2>

Let's see our model predicting some images it never saw

In [ ]:
# scracthes_test_sample = r'..\test\scratches\scratches_10.jpg'
# patches_test_sample = r'..\test\patches\patches_10.jpg'
# inclusion_test_sample = r'..\test\inclusion\inclusion_10.jpg'

# def image_and_pred(model, img_dir):
#     img = Image.open(img_dir)
#     img = transforms(img)
#     img = img.to(device)
#     img = img.unsqueeze(0)
#     prediction = model(img) # saved model is on gpu, no need to model.to(device)
#     prediction = prediction.argmax()
#     print('Predicted label: ', categories[prediction])
#     process(img_dir)

And here is the performance on whole test data

In [31]:
test_loss = 0
total = 0
correct = 0
acc = list()
for inputs, targets in test_data_loader:
    inputs = inputs.to(device)
    output = cnn(inputs)
    targets = targets.to(device)
    batch_loss = torch.nn.CrossEntropyLoss()(output, targets)
    test_loss += batch_loss.data.item() # cumulatively adding losses on batches
    _, y_pred_tags = output.max(1)
    total += targets.size(0)
    correct += y_pred_tags.eq(targets).sum().item()
    acc.append(100.*correct/total) # accuracy on a batch

test_loss /= len(test_data_loader) # total test loss
avg_acc = np.mean(acc)

print('Test loss: {:.2f}, Avg accuracy: {:.2f}'.format(test_loss, avg_acc)) 

Test loss: 0.15, Avg accuracy: 97.68
